# lyrics2vec 
## Passo-a-passo para reproduzir os resultados

Lyrics2vec é um experimento para visualizar vetores de músicas de artistas. Vamos usar doc2vec para os vetores para poder usar os títulos das canções como tags associadas para cada palavra da música correspondente.

As tags em Doc2Vec não correspondem a clusters de palavras mas sim a vetores especiais daquela tag associados à palavras específicas. Gosto de pensar nas tags como meta-vetores. A imagem abaixo ilustra bem isso, retirada do [fórum do gensim](https://groups.google.com/d/msg/gensim/EwK-6JgkWVI/yO4yzgkGCAAJ).

![](http://picload.org/image/paoaocc/doc2vec.png)

Sem mais delongas, vamos começar nosso experimento!

---

Esta iniciativa foi inspirada por este [medium](https://medium.com/towards-data-science/we-used-data-science-to-explore-ed-sheerans-songs-here-s-what-we-found-480b56b23517#.j8up1oyqr).

## Passo 1: Configuração e Obtenção dos Dados

Vamos começar importando alguns módulos e configurando. Precisaremos de:

- **pandas** para manipulação dos dados
- **gensim** para montar o modelo doc2vec
- **multiprocessing** para usar todos núcleos da cpu
- **robobrowser** para raspagem de dados
- **pprint** para melhor exibir os resultados.

Obteremos os dados do [MLDb](http://mldb.org).

É preciso habilitar o histórico no RoboBrowser para melhor navegação pela lista de músicas e configurar um timeout alto pois o mldb.org apresenta alguns erros de timeout de vez em quando.

In [39]:
import pandas as pd
import gensim
import multiprocessing
from robobrowser import RoboBrowser
import pprint

In [28]:
workers = multiprocessing.cpu_count()
browser = RoboBrowser(history=True, timeout=300)

Agora, vamos capturar as músicas e armazenar os dados em duas variáveis. **songs_lyrics** vai armazenar as letras das músicas e **songs_lyrics_titles**, os títulos das músicas. Isso será necessário mais a frente quando utilizaremos os títulos das músicas como nossas tags.

Uma breve explicação sobre o RoboBrowser no código abaixo:

- **browser.open**: acessa o site que desejamos, um site com a lista de músicas de determinado artista
- **browser.select**: seleciona o elemento desejado, retorna uma lista com o conteúdo deste elemento
- **browser.follow_link**: acessa o link contido no elemento selecionado
- **browser.back**: nos leva de volta para a lista de músicas

Com isso, extraimos os dados que queremos. Pode demorar um pouco para processar tudo. Separei em duas células para que possamos ver melhor o resultado de cada etapa.

In [30]:
browser.open('http://www.mldb.org/artist-39-the-beatles.html')
songs = browser.select('#thelist td a')
songs_lyrics =[]
for song in songs:
    browser.follow_link(song)
    lyrics = browser.select('.songtext')
    lyrics_text = lyrics[0].text
    songs_lyrics.append(lyrics_text)
    browser.back
print(songs_lyrics)

/usr/lib/python3/dist-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


["It's been a hard day's night \nAnd I've been working like a dog \nIt's been a hard day's night \nI should be sleeping like a log \nBut when I get home to you \nI find the things that you do \nWill make me feel alright \n\nYou know I work all day \nTo get you money to buy you things \nAnd it's worth it just to hear you say \nYou're gonna give me everything \nSo why on earth should I moan \n'Cause when I get you alone \nYou know I feel okay \n\nWhen I'm home everything seems to be right \nWhen I'm home feeling you holding me tight, tight, yeah \n\nIt's been a hard day's night \nAnd I've been working like a dog \nIt's been a hard day's night \nI should be sleeping like a log \nBut when I get home to you \nI find the things that you do \nWill make me feel alright \n\nSo why on earth should I moan \n'Cause when I get you alone \nYou know I feel okay \n\nWhen I'm home everything seems to be right \nWhen I'm home feeling you holding me tight, tight, yeah \n\nIt's been a hard day's night \nA

In [31]:
browser.open('http://www.mldb.org/artist-39-the-beatles.html')
song_titles = browser.select('#thelist td a')
songs_lyrics_title =[]
for song_title in song_titles:
    lyrics_title = song_title.text
    songs_lyrics_title.append(lyrics_title)
    browser.back
print(songs_lyrics_title)

/usr/lib/python3/dist-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


["A Hard Day's Night", 'All You Need Is Love', "Can't Buy Me Love", 'Come Together', 'Day Tripper', 'Eight Days A Week', 'Eleanor Rigby', 'From Me To You', 'Get Back', 'Hello, Goodbye', 'Help!', 'Hey Jude', 'I Feel Fine', 'I Want To Hold Your Hand', 'Lady Madonna', 'Let It Be', 'Love Me Do', 'Paperback Writer', 'Penny Lane', 'She Loves You', 'Something', 'The Ballad Of John And Yoko', 'The Long And Winding Road', 'Ticket To Ride', 'We Can Work It Out', 'Yellow Submarine', 'Yesterday', "A Hard Day's Night", 'And I Love Her', 'Any Time At All', "Can't Buy Me Love", 'I Should Have Known Better', "I'll Be Back", "I'll Cry Instead", "I'm Happy Just To Dance With You", 'If I Fell', 'Tell Me Why', 'Things We Said Today', 'When I Get Home', "You Can't Do That", 'Because', 'Carry That Weight', 'Come Together', 'Golden Slumbers', 'Here Comes The Sun', "I Want You ( She's So Heavy )", "Maxwell's Silver Hammer", 'Mean Mr. Mustard', "Octopus's Garden", 'Oh! Darling', 'Polythene Pam', 'She Came In T

Uma vez de posse dos dados, precisamos manipular para obter um formato que possamos usar no Doc2Vec. São duas etapas desse pré-processamento. Primeiro vamos unir as variáveis em uma lista de lista que chamaremos de **songs_list**.

Para criar essa songs_list, vamos usar pandas. Primeiros associamos as listas em um dicionário que chamamos **data** e criamos um DataFrame a partir deste dicionário. Com a função drop_duplicates(), fazemos uma limpeza dos dados reduzindo de 326 músicas para 240 músicas.

Após esse rápido processamento, transformamos os dados de volta para lista com a função to_list() do pandas. Ao fim dessa etapa teremos unido as duas variáveis em uma única variável.

In [32]:
data = {'title' : songs_lyrics_title, 'lyrics' : songs_lyrics}
df = pd.DataFrame(data)
unique_songs = df.drop_duplicates(subset='title')

In [33]:
songs_list = unique_songs.values.tolist()
songs_list

[["It's been a hard day's night \nAnd I've been working like a dog \nIt's been a hard day's night \nI should be sleeping like a log \nBut when I get home to you \nI find the things that you do \nWill make me feel alright \n\nYou know I work all day \nTo get you money to buy you things \nAnd it's worth it just to hear you say \nYou're gonna give me everything \nSo why on earth should I moan \n'Cause when I get you alone \nYou know I feel okay \n\nWhen I'm home everything seems to be right \nWhen I'm home feeling you holding me tight, tight, yeah \n\nIt's been a hard day's night \nAnd I've been working like a dog \nIt's been a hard day's night \nI should be sleeping like a log \nBut when I get home to you \nI find the things that you do \nWill make me feel alright \n\nSo why on earth should I moan \n'Cause when I get you alone \nYou know I feel okay \n\nWhen I'm home everything seems to be right \nWhen I'm home feeling you holding me tight, tight, yeah \n\nIt's been a hard day's night \n

Última etapa de pré-processamento. Agora só precisamos iterar na nossa lista songs_list para gerar um formato que é aceito pelo Doc2Vec. As palavras devem ser tokenizadas e label declarada explicitamente no seguinte formato:

    LabeledSentence(words=['várias', 'palavras', 'diferentes'], tags=[label])

Uma maneira rápida de gerar os tokens de palavras é com a função simple_preprocess do gensim. Essa função já remove muitos ruídos. Você pode usar outras formas também caso queiro incluir pontuação, por exemplo, nos seus dados.

Atribuímos os resultados a variável **sentences**. O nome é apenas uma conveniência, pois cada item da lista não corresponde a orações de fato, mas sim a letra da música inteira. Poderíamos nomear de qualquer outra forma.

In [60]:
sentences=[]

for item in songs_list:
    
    features = [item[0]]
    labels = [item[1]]
    for feature in features:
        features_ready = gensim.utils.simple_preprocess(feature)
        for label in labels:
            sentences.append(gensim.models.doc2vec.LabeledSentence(words = features_ready, tags = [label]))
print(sentences)

[LabeledSentence(words=['it', 'been', 'hard', 'day', 'night', 'and', 've', 'been', 'working', 'like', 'dog', 'it', 'been', 'hard', 'day', 'night', 'should', 'be', 'sleeping', 'like', 'log', 'but', 'when', 'get', 'home', 'to', 'you', 'find', 'the', 'things', 'that', 'you', 'do', 'will', 'make', 'me', 'feel', 'alright', 'you', 'know', 'work', 'all', 'day', 'to', 'get', 'you', 'money', 'to', 'buy', 'you', 'things', 'and', 'it', 'worth', 'it', 'just', 'to', 'hear', 'you', 'say', 'you', 're', 'gonna', 'give', 'me', 'everything', 'so', 'why', 'on', 'earth', 'should', 'moan', 'cause', 'when', 'get', 'you', 'alone', 'you', 'know', 'feel', 'okay', 'when', 'home', 'everything', 'seems', 'to', 'be', 'right', 'when', 'home', 'feeling', 'you', 'holding', 'me', 'tight', 'tight', 'yeah', 'it', 'been', 'hard', 'day', 'night', 'and', 've', 'been', 'working', 'like', 'dog', 'it', 'been', 'hard', 'day', 'night', 'should', 'be', 'sleeping', 'like', 'log', 'but', 'when', 'get', 'home', 'to', 'you', 'find',

## Treinamento

Na etapa de treinamento, reuní vários acontecimentos importantes em apenas duas células. Vamos ver com calma o que acontece em cada linha de código agora.

Primeiro quebramos a lista **sentences** para alimentar no treinamento do Doc2Vec. Depois criamos a variável model com os parâmetros que usaremos para treinamento. Aqui ainda não acontece nenhum treinamento de fato. Vamos entender os parâmetros:

- **size**: quantas dimensões cada vetor terá
- **min_count**: palavras abaixo dessa quantidade são desconsideradas. Precisamos deixar 1 para contabilizar as labels
- **window**: quantas palavras antes e depois de cada palavra-alvo o algoritmo deve considerar como contexto
- **iter**: quantas vezes ler todo o dataset de entrada
- **alpha e min_alpha**: quoeficientes de aprendizado
- **workers**: quantos núcleos usar para o processamento. Já calculado pelo módulo multiprocessing.

Em seguida precisamos criar um vocabulário para nosso modelo. Esta etapa é essencial.

Para ter certeza que tudo está dando certo até aqui, vamos calcular a memória estimada e ver os resultados.

In [61]:
train_corpus = list(sentences)
model = gensim.models.doc2vec.Doc2Vec(size=50, min_count=1, window=10, iter=100, alpha=0.025, min_alpha=0.025, workers=workers)
model.build_vocab(train_corpus)
model.estimate_memory()

{'doctag_lookup': 48000,
 'doctag_syn0': 48000,
 'syn0': 493800,
 'syn1neg': 493800,
 'total': 2318100,
 'vocab': 1234500}

Hora do treinamento de verdade. Esta etapa, pode parecer um pouco complicada. Seguimos as instruções do tutorial do Radim, o autor do gensim de como fazer para controlar a taxa de aprendizado de forma que não percamos informações durante esse processo.

O Model.train é onde a "mágica" acontece e para cada iteração iremos redefinir a taxa de aprendizado. Vamos usar o **%%time** do jupyter para ver quanto tempo leva de processamento.

In [65]:
%%time 
for epoch in range(100):
    model.train(train_corpus)
    model.alpha -= 0.002
    model.min_alpha = model.alpha

CPU times: user 9min 24s, sys: 32.7 s, total: 9min 57s
Wall time: 3min 2s


## Hora dos Testes

Agora que temos nosso modelo treinado e pronto para uso vamos testá-lo. As mesmas funçoes disponíveis para Word2Vec também podem ser usadas aqui com algumas funções a mais. Vamos revisar algumas operações interessantes que podemos fazer com vetores de palavras.

- **model['palavra']**: retorna a matriz do vetor que representa aquela palavra
    - array([-0.00449447, -0.00310097,  0.02421786, ...], dtype=float32)
- **model.doesnt_match("almoço lanche jantar futebol".split())**: retorna a palavra que não pertence ao grupo, isto é, o vetor mais distante
    - futebol
- **model.similarity('palavra1', 'palavra2')**: retorna a similaridade entre dois vetores
    - 0.73723527
- **model.most_similar(positive=['mulher','rei'],negative=['homem'])**: calcula uma analogia entre vetores, pode ser usado também para encontrar a similaridade de um único vetor como em most_similar('palavra').
    - rainha
    
As mesmas operações podem ser usadas para os vetores das tags de documentos. Para isso basta usar model.docvecs.função. Exemplo:

    model.docvecs.most_similar("Label")
    
Além disso, o Doc2Vec também possui a função infer_vectors() que permite inferir vetores a partir de uma nova string não vista durante o treinamento.

Vamos ver esses exemplos na prática.

In [66]:
model.docvecs.most_similar("Let It Be")

[("Don't Let Me Down", 0.6624167561531067),
 ('Polythene Pam', 0.6284393072128296),
 ('Being For The Benefit Of Mr. Kite', 0.6186915040016174),
 ('You Know My Name( Up The Number)', 0.6029567122459412),
 ('Dear Prudence', 0.5618879795074463),
 ('Ob-La-Di, Ob-La-Da', 0.5589374899864197),
 ('We Can Work It Out', 0.5449695587158203),
 ('Helter Skelter', 0.5370487570762634),
 ('Maggie May', 0.5286579728126526),
 ('I Am The Walrus', 0.5205299258232117)]

In [67]:
vec = [model.docvecs["Let It Be"] - model["trouble"]]
[m for m in model.docvecs.most_similar(vec, topn=11) if m[0] != "Let It Be"]

[("Don't Let Me Down", 0.6450002789497375),
 ('Polythene Pam', 0.6151739358901978),
 ('Being For The Benefit Of Mr. Kite', 0.6096665859222412),
 ('You Know My Name( Up The Number)', 0.6011987328529358),
 ('We Can Work It Out', 0.5552796125411987),
 ('Helter Skelter', 0.5532039403915405),
 ('Ob-La-Di, Ob-La-Da', 0.5506325364112854),
 ('Maggie May', 0.5407522916793823),
 ('Dear Prudence', 0.5392324328422546),
 ('Tell Me Why', 0.5173360705375671)]

Nos exemplos acima, encontramos músicas similares a Let It Be e a distância entre seus respectivos vetores. Em seguida, como exercício, subtraímos o vetor trouble presente na música Let it Be nos versos "when I find myself in times of trouble" e encontramos não só algumas músicas diferentes como a distância entre as músicas similares encontradas anteriormente foi modificada por esta operação.

Vamos agora encontrar a similaridade entre algumas músicas

In [68]:
model.docvecs.doesnt_match(['Love Me Do', 'Yesterday', 'Let It Be', 'Lucy In The Sky With Diamonds'])

'Let It Be'

In [76]:
model.docvecs.similarity('Being For The Benefit Of Mr. Kite', 'Help!')

0.62093688843392403

## Conclusão

Os vetores das letras de músicas servem para identificar similaridades semânticas e sintáticas entre estas músicas pois analisa o contexto em que cada palavra aparece nos documentos.

O modelo pode ser treinado com parâmetros diferentes. Mais testes são necessários para identificar os parâmetros ideais. Talvez para artistas diferentes sejam necessários parâmetros diferentes devido a estilos individuais de métricas poéticas, vocabulário usado e quantidade de músicas publicadas.

Uma vez satisfeitos com o modelo podemos salvá-lo com o comando abaixo

    model.save('path/to/model.vectors')
    
e depois podemos carregar o mesmo modelo pré-treinado com

    model = gensim.models.Doc2Vec.load('path/to/model.vectors')
    
Numa próxima etapa deste exercício iremos visualizar os vetores em gráficos indicando as relações semânticas entre as músicas.